In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Make identity block

In [2]:
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.add([x, inputs])
        x = self.relu(x)
        
        return x

# Make ResNet

In [7]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 3, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((2, 2))
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = self.id1a(x)
        x = self.id1b(x)
        x = self.global_pool(x)
        return self.classifier(x)

In [12]:
from tensorflow.keras.datasets import mnist, cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.
x_test = x_test / 255.

resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['acc'])
resnet.fit(x_train, y_train, epochs=5)
resnet.evaluate(x_test, y_test)

Epoch 1/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.3212 - acc: 0.5225
Epoch 2/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0012 - acc: 0.6459
Epoch 3/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.8667 - acc: 0.6944
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.7762 - acc: 0.7289
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.9316 - acc: 0.6800


[0.931609570980072, 0.6800000071525574]